In [1]:
import numpy as np
from random import shuffle
from sys import exit

In [2]:
global card_per_turn

card_per_player = {2:7, 3:6, 4:6, 5:6}
players = 2
card_per_turn = 2
danger_thresh = 10
one_more_tresh = 2

global N,m
N = players

n = 2 #card_per_turn)
m = card_per_player[players]

In [3]:
def pick_one(hand,n):
    if deck == []:
        n = card_per_turn - 1
        print("-------------------------------Deck is over, last few rounds-------------------------------")
    if deck == []:
        return hand,n
    hand.append(deck[0])
    deck.pop(0)
    return hand,n

In [4]:
def scan_hand(hand,discards=[],obligatory_move=True):
# delta is the distance of the bast match
# ind contains first the in hand index then the pile index
    tops = [pile[-1] for pile in piles]
    delta = 100
    ind = [0,0]
    for i, hand_i in enumerate(hand):
        for j, top_j in enumerate(tops):
            if j in discards: continue
            old_delta = delta
            old_ind = ind
            delta = hand_i - top_j
            if j > 1: delta = -delta
            ind = [i,j]
            if delta == -10:
                return delta, ind
            elif delta >= old_delta or delta < 0 :
                delta = old_delta
                ind = old_ind
    if delta==100 and discards==[] and obligatory_move: end_game_and_recap() #replace this with exit program command when the code is implemented in a script
    return delta, ind

In [5]:
def print_piles(piles):
    # Find the maximum length among all lists
    max_len = max(len(pile) for pile in piles)
    
    # Print the piles in columns
    for i in range(max_len):
        row = []
        for pile in piles:
            # If the pile still has elements at this index, add the element
            if i < len(pile):
                row.append(str(pile[i]))
            else:
                row.append("")  # Empty space if the pile is finished
        
        # Print the row with uniform spacing
        print(f"{row[0]:<4} {row[1]:<4} {row[2]:<4} {row[3]:<4}")

In [6]:
def end_game_and_recap():
    #print_piles(piles)
    raise ValueError("Game Over")

In [7]:
#interaction(hands[inactive_players])
def interaction(hands):
    #things to implement: give higher priority to closest players
    n_hands = len(hands)
    delta_list = []
    pile_idx_list = []
    for i in range(n_hands):
        delta,(_,pile_idx)=scan_hand(hands[i])
        pile_idx_list.append(pile_idx)
        delta_list.append(delta)
    return list(set(pile_idx_list))

In [8]:
def place_card(hand,card_idx,pile_idx):
    print('Card placed: ', hand[card_idx], 'on pile ',pile_idx)
    piles[pile_idx].append(hand[card_idx]) #add played card to the piles
    hand.pop(card_idx) #remove card played from the hand
    return hand

In [9]:
def play_card(hand,player_idx,obligatory_move=True):
    delta_1,(card_idx1,pile_idx1)=scan_hand(hand,obligatory_move=obligatory_move)
    if delta_1==-10:
        hand=place_card(hand,card_idx1,pile_idx1)
    elif not obligatory_move and delta_1>one_more_tresh:
        return hand
    else:
        #mask_inactive = [i!=player_idx for i in range(N)]
        hands_non_active = [hand for i,hand in enumerate(hands) if not i==player_idx] #np.asarray(hands)[mask_inactive].tolist()
        dont_pile_idx_list = interaction(hands_non_active)
        
        if not pile_idx1 in dont_pile_idx_list: #case 1: the pile of interest can be played
            hand=place_card(hand,card_idx1,pile_idx1)
        elif dont_pile_idx_list==[0,1,2,3]: #case 2: all piles are taken, so fuck it
            hand=place_card(hand,card_idx1,pile_idx1)
        else: #case 3: check other piles
            delta_2,(card_idx2,pile_idx2)=scan_hand(hand,discards=dont_pile_idx_list,obligatory_move=obligatory_move)
            if delta_2-delta_1>danger_thresh or delta_2==100:
                hand=place_card(hand,card_idx1,pile_idx1)
            else:
                if not obligatory_move and delta_2>one_more_tresh:
                    return hand
                else: hand=place_card(hand,card_idx2,pile_idx2)
    return hand    

In [10]:
def play_turn(player_idx,n):
    n_played=n   #this is incremented only in case one or more non-obbligatory cards are played
    hand=hands[player_idx]

    print('player ', player_idx,' , hand: ',hand)
    print_piles(piles)
    
    for _ in range(n):
        hand=play_card(hand,player_idx)

    while True:
        hand_old=hand.copy()
        hand=play_card(hand,player_idx,obligatory_move=False)
        #print(hand,hand_old)
        if hand==hand_old: break
        n_played = n_played + 1
        
    print('n. played cards: ',n_played,'\n')
    for _ in range(n_played):
        hand,n=pick_one(hand,n)
    hands[player_idx]=hand
    return n

In [11]:
global deck, piles
deck = [int(i) for i in np.arange(2,99)]
shuffle(deck)
piles = [[1],[1],[100],[100]]

hands = np.asarray(deck[:N*m]).reshape([N,m]).tolist()
deck = deck[N*m:]
#Convention: decreasing piles start with 100 but for sake of ease
# we consider them at -100 so that they are increasing
# in this way we simply change the sign for cards in hands

n = 2 #card_per_turn)

while True:
    for player_idx in range(N):
        n=play_turn(player_idx,n)
        if hands==[[]*N]: break
print('Win')

player  0  , hand:  [86, 29, 6, 42, 88, 46, 92]
1    1    100  100 
Card placed:  6 on pile  1
Card placed:  92 on pile  2
n. played cards:  2 

player  1  , hand:  [7, 10, 49, 61, 66, 63, 25]
1    1    100  100 
     6    92       
Card placed:  7 on pile  1
Card placed:  10 on pile  1
n. played cards:  2 

player  0  , hand:  [86, 29, 42, 88, 46, 18, 84]
1    1    100  100 
     6    92       
     7             
     10            
Card placed:  88 on pile  2
Card placed:  86 on pile  2
Card placed:  84 on pile  2
n. played cards:  3 

player  1  , hand:  [49, 61, 66, 63, 25, 76, 3]
1    1    100  100 
     6    92       
     7    88       
     10   86       
          84       
Card placed:  3 on pile  0
Card placed:  76 on pile  2
n. played cards:  2 

player  0  , hand:  [29, 42, 46, 18, 70, 16, 87]
1    1    100  100 
3    6    92       
     7    88       
     10   86       
          84       
          76       
Card placed:  16 on pile  1
Card placed:  70 on pile  2
Card 

ValueError: Game Over